# Prepatation

## Instaling Libararies

In [2]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import matplotlib.pyplot as plt
import datetime
import yfinance as yf
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
from scipy.optimize import minimize
import pytz
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import STL
import seaborn as sns
# from sklearn.model_selection import TimeSeriesSplit
import statsmodels.tsa.api as smt
from copy import deepcopy
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import arma_order_select_ic
import warnings
warnings.filterwarnings("ignore")
import statsmodels.tools.eval_measures
import scipy 
from scipy import stats
import time
from matplotlib.dates import DateFormatter
from matplotlib.dates import YearLocator, DateFormatter
from statsmodels.tsa.api import VAR

In [9]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

## Inporting data

In [3]:
df_main = pd.read_excel('./RAW_DATA.xlsx')


# Data Formatting

In [10]:
df_main

,Unnamed: 0,BTCUSD,Unnamed: 2,DXY,Unnamed: 4,SPX
0,2014-09-17,457.334015,2014-09-17,84.699997,2014-09-17,2001.57
1,2014-09-18,424.440002,2014-09-18,84.320000,2014-09-18,2011.36
2,2014-09-19,394.795990,2014-09-19,84.800003,2014-09-19,2010.4
3,2014-09-20,408.903992,2014-09-21,NaN,2014-09-22,1994.29
4,2014-09-21,398.821014,2014-09-22,84.669998,2014-09-23,1982.77
...,...,...,...,...,...,...
3332,2023-11-01,35437.253906,NaT,NaN,NaT,NaN
3333,2023-11-02,34938.242188,NaT,NaN,NaT,NaN
3334,2023-11-03,34732.324219,NaT,NaN,NaT,NaN
3335,2023-11-04,35082.195313,NaT,NaN,NaT,NaN


In [5]:
df_main.head(10)

,Unnamed: 0,BTCUSD,Unnamed: 2,DXY,Unnamed: 4,SPX
0,2014-09-17,457.334015,2014-09-17,84.699997,2014-09-17,2001.57
1,2014-09-18,424.440002,2014-09-18,84.320000,2014-09-18,2011.36
2,2014-09-19,394.795990,2014-09-19,84.800003,2014-09-19,2010.4
3,2014-09-20,408.903992,2014-09-21,NaN,2014-09-22,1994.29
4,2014-09-21,398.821014,2014-09-22,84.669998,2014-09-23,1982.77
5,2014-09-22,402.152008,2014-09-23,84.660004,2014-09-24,1998.3
6,2014-09-23,435.790985,2014-09-24,85.050003,2014-09-25,1965.99
7,2014-09-24,423.204987,2014-09-25,85.199997,2014-09-26,1982.85
8,2014-09-25,411.574005,2014-09-26,85.639999,2014-09-29,1977.8
9,2014-09-26,404.424988,2014-09-28,NaN,2014-09-30,1972.29


In [16]:
# 元のデータフレームを作成
df_original = pd.DataFrame(df_main)

# インデックスとして使う日付の範囲を作成
date_range = pd.date_range(start='2014-09-17', end=pd.to_datetime('today'))

# インデックス用の空のデータフレームを作成
df_indexed = pd.DataFrame(index=date_range)

# 元データから新しいデータフレームに日付に対応するデータをマッピング
df_indexed['BTCUSD'] = df_original.set_index('Unnamed: 0')['BTCUSD']
df_indexed['DXY'] = df_original.set_index('Unnamed: 2')['DXY']
df_indexed['SPX'] = df_original.set_index('Unnamed: 4')['SPX']

# NaNをNoneに変換
df_indexed = df_indexed.where(pd.notnull(df_indexed), None)

print(df_indexed)

ValueError: cannot reindex on an axis with duplicate labels

In [15]:
df = df_main.copy()

# 'Date' 列を作成し、最初の日付の列の値を代入します。
df['Date'] = df['Unnamed: 0']

# 新たな日付列で対応する日付がない場合はNoneを代入します。
df['BTCUSD'] = np.where(df['Date'] == df['Unnamed: 0'], df['BTCUSD'], None)
df['DXY'] = np.where(df['Date'] == df['Unnamed: 2'], df['DXY'], None)
df['SPX'] = np.where(df['Date'] == df['Unnamed: 4'], df['SPX'], None)

# 'Date' 列をインデックスに設定します。
df.set_index('Date', inplace=True)

# 不要な列を削除します。
df.drop(columns=['Unnamed: 0', 'Unnamed: 2', 'Unnamed: 4'], inplace=True)

# 結果を表示します。
print(df)

                  BTCUSD        DXY      SPX
Date                                        
2014-09-17    457.334015  84.699997  2001.57
2014-09-18    424.440002      84.32  2011.36
2014-09-19     394.79599  84.800003   2010.4
2014-09-20    408.903992       None     None
2014-09-21    398.821014       None     None
...                  ...        ...      ...
2023-11-01  35437.253906       None     None
2023-11-02  34938.242188       None     None
2023-11-03  34732.324219       None     None
2023-11-04  35082.195313       None     None
2023-11-05  35081.945313       None     None

[3337 rows x 3 columns]


In [13]:
df.head(100)

,BTCUSD,DXY,SPX
Date,,,
2014-09-17,457.334015,84.699997,2001.57
2014-09-18,424.440002,84.32,2011.36
2014-09-19,394.79599,84.800003,2010.4
2014-09-20,408.903992,None,None
2014-09-21,398.821014,None,None
2014-09-22,402.152008,None,None
2014-09-23,435.790985,None,None
2014-09-24,423.204987,None,None
2014-09-25,411.574005,None,None


# Data Analysis